In [121]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [122]:
#reading data from CSV file
dataset = pd.read_csv('D:\ELLAssDataset\health_data.csv')
dataset  = dataset.sample(frac = 1)
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1:].values
datasize = X.shape[0]
X_train = X[:(datasize*7)//10,:]
y_train = y[:(datasize*7)//10,:]
X_test = X[(datasize*7)//10:,:]
y_test =  y[(datasize*7)//10:,:]
print(dataset)

     age  restbps  chol  category
445   54      147   304         1
442   55      145   318         1
64    28      109   240         0
102   23      102   234         0
512   61      140   372         1
..   ...      ...   ...       ...
104   27       99   258         0
371   42      134   153         0
207   34      115   264         0
517   62      147   328         1
273   36      109   167         0

[700 rows x 4 columns]


In [123]:
X_train_0 = X_train[y_train[:,0]==0]
X_train_1 = X_train[y_train[:,0]==1]
print(X_train_1)

[[ 54 147 304]
 [ 55 145 318]
 [ 61 140 372]
 [ 57 160 348]
 [ 60 156 234]
 [ 67 153 364]
 [ 63 137 343]
 [ 60 142 357]
 [ 47 135 214]
 [ 56 147 397]
 [ 47 138 217]
 [ 66 147 351]
 [ 66 135 351]
 [ 41 137 262]
 [ 48 138 243]
 [ 40 136 234]
 [ 72 137 310]
 [ 57 137 277]
 [ 36 134 223]
 [ 65 137 370]
 [ 55 152 234]
 [ 58 166 302]
 [ 58 143 366]
 [ 63 145 367]
 [ 49 125 164]
 [ 46 130 331]
 [ 63 139 431]
 [ 43 126 266]
 [ 47 139 259]
 [ 64 151 355]
 [ 45 120 218]
 [ 34 135 251]
 [ 44 119 299]
 [ 50 141 270]
 [ 39 131 239]
 [ 59 173 293]
 [ 45 137 214]
 [ 44 146 204]
 [ 63 172 388]
 [ 60 151 370]
 [ 47 128 265]
 [ 58 156 332]
 [ 45 140 229]
 [ 51 140 319]
 [ 60 148 276]
 [ 59 143 318]
 [ 74 133 324]
 [ 53 135 274]
 [ 47 131 170]
 [ 67 134 324]
 [ 61 139 392]
 [ 67 135 302]
 [ 46 151 322]
 [ 69 143 333]
 [ 49 119 194]
 [ 66 145 348]
 [ 64 136 297]
 [ 58 146 290]
 [ 67 158 395]
 [ 55 161 309]
 [ 55 168 280]
 [ 45 124 160]
 [ 42 116 224]
 [ 34 119 186]
 [ 73 141 383]
 [ 64 126 359]
 [ 64 148 

In [124]:
#Multivariate gaussian distribution
def Multivariate_gaussian(inp,mean,covariance):
    det_covariance = np.linalg.det(covariance)
    front = ((2*np.pi)**3)*det_covariance
    front = pow(front, 0.5)
    front = 1/front
    exp = np.matmul(np.transpose(inp - mean),np.linalg.inv(covariance))
    exp = np.matmul(exp,inp-mean)
    exp = np.exp(-0.5*exp)
    ret = front*exp
    return ret

In [125]:
#Calculating the mean and Variance for the class 0
N_0 = X_train_0.shape[0]
mean_0 = (1/N_0)*np.sum(X_train_0, axis = 0)
print(mean_0)
covariance_0 = (1/N_0)*(np.matmul(np.transpose(X_train_0-mean_0),X_train_0 - mean_0))
print(covariance_0)

[ 32.39711191 114.01083032 231.41516245]
[[ 91.12750068  99.44335258 -16.19735693]
 [ 99.44335258 166.92768054 -25.49908118]
 [-16.19735693 -25.49908118 875.21753183]]


In [126]:
#Calculating the mean and covariance matrix for the class 1
N_1 = X_train_1.shape[0]
mean_1 = (1/N_1)*np.sum(X_train_1, axis = 0)
covariance_1 = (1/N_1)*(np.matmul(np.transpose(X_train_1-mean_1),X_train_1 - mean_1))
print(mean_1)
print(covariance_1)

[ 54.90610329 141.90140845 291.1314554 ]
[[  92.83625383   41.49778483  434.82924464]
 [  41.49778483  142.3048337   289.25709185]
 [ 434.82924464  289.25709185 3771.87004342]]


In [127]:
def predict(inp):
    p_0 = Multivariate_gaussian(inp,mean_0,covariance_0)
    p_1 = Multivariate_gaussian(inp,mean_1,covariance_1)
    if p_0 > p_1:
        return 0
    else:
        return 1

In [128]:
def Accuracy_of_classifier(test_X,test_y):
    count = 0
    score = [[0,0] for i in range(2)]
    for i in range(test_X.shape[0]):
        pred_cat = predict(test_X[i])
        score[test_y[i][0]][pred_cat]+=1
        if test_y[i][0]==pred_cat:
            count+=1
    print(score)
    precision = score[1][1]/(score[0][1]+score[1][1])
    recall = score[1][1]/(score[1][0]+score[1][1])
    F1 = (2*precision*recall)/(precision+recall)
    print("Accuracy: {} %".format(count*100/len(test_X)))
    print("Precision: {}".format(precision))
    print("Recall: {}".format(recall))
    print("F1 Score: {}".format(F1))


In [129]:
Accuracy_of_classifier(X_train,y_train)

[[235, 42], [29, 184]]
Accuracy: 85.51020408163265 %
Precision: 0.8141592920353983
Recall: 0.863849765258216
F1 Score: 0.8382687927107062


In [130]:
Accuracy_of_classifier(X_test, y_test)

[[110, 13], [16, 71]]
Accuracy: 86.19047619047619 %
Precision: 0.8452380952380952
Recall: 0.8160919540229885
F1 Score: 0.8304093567251463
